## DEV - *sim* Spot Detection

### Notes

- Outline of approach
    1. Adaptive background subtraction using heavily Gaussian-smoothed background
    2. Spot detection with `skimage.feature.blob_log`
    4. Compute various measurements

### Prep

In [ ]:
# Imports

import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndi

from skimage import io
from skimage.feature import blob_log
from skimage.filters import threshold_li

from ipywidgets import interact

In [ ]:
# Input file path for test file

fpath = r'..\Data\Images\Sim\Continuous\1_early_sim timeseries stacks_zmax.tif'

In [ ]:
# Load time course

img = io.imread(fpath)
print(img.shape, img.dtype)

In [ ]:
# Show time course

@interact(t=(0,img.shape[0]-1,1))
def show_timecourse(t=0, show=False):
    if show:
        fig = plt.figure(figsize=(12,12))
        plt.imshow(img[t], cmap='gray')
        plt.title('raw')
        plt.show()        

### Adaptive Background Subtraction

Note: I've tried doing some Gaussian smoothing prior to this, but it didn't make a difference or was even a bit detrimental to the spot detection.

In [ ]:
# Apply background subtraction

bg = np.array([ndi.gaussian_filter(img[z], 10) for z in range(img.shape[0])])
img_bgsub = img - bg
img_bgsub[img < bg] = 0

In [ ]:
# Show results

@interact(t=(0,img.shape[0]-1,1))
def show_timecourse(t=0, show=False):
    if show:
        plt.figure(figsize=(12,12))
        plt.imshow(img_bgsub[t], cmap='gray')
        plt.title('bgsub')
        plt.show()

### Spot Detection

In [ ]:
# Initial test of blob_log

# Try it
test_t = 33
blobs_test = blob_log(img_bgsub[test_t], min_sigma=0.7, max_sigma=5, overlap=0.0)

# Show the result
plt.figure(figsize=(12,12))
plt.imshow(img_bgsub[test_t], cmap='gray')
plt.scatter(blobs_test[:,1], blobs_test[:,0], marker='o', facecolor='none', edgecolor='r')
plt.title('blob_log')
plt.show()

In [ ]:
# Elbow plot to determine min_sigma

# Run it
counts = []
min_sigma_space = np.linspace(0.1, 1.5, 40)
for min_sigma in min_sigma_space:
    blobs_elbow = blob_log(img_bgsub[test_t], min_sigma=min_sigma, max_sigma=5, overlap=0)
    counts.append(blobs_elbow.shape[0])
    
# Show it
plt.plot(min_sigma_space, counts)
plt.xlabel('min_sigma')
plt.ylabel('spots detected')
plt.xlim(0.1, 1.0)
plt.show()

# ->> Detection is robust at min_sigma >= 0.6; using 0.7

In [ ]:
# Detect blobs across time course

blobs = [blob_log(img_bgsub[t], min_sigma=0.7, max_sigma=5) for t in range(img.shape[0])]

In [ ]:
# Show result

@interact(t=(0,img.shape[0]-1,1))
def show_timecourse(t=0, show=False):
    if show:
        plt.figure(figsize=(12,12))
        plt.imshow(img_bgsub[t], cmap='gray')
        plt.scatter(blobs[t][:,1], blobs[t][:,0], marker='o', facecolor='none', edgecolor='r')
        plt.title('bgsub')
        plt.show()

### Compute Measurements

#### Create Bounding Boxes

In [ ]:
# Test out bounding box cutting

# Select test spot
test_t = 33
test_spot = 120

# Get test spot
coords = blobs[test_t][test_spot][:2]

# Get size as 3*sigma, either individual or aggregate
size = 3 * blobs[test_t][test_spot][2]
#size = 3 * np.percentile(np.concatenate(blobs)[:,2], 90)
#size = 3 * np.median(np.concatenate(blobs)[:,2])

# Create bounding box
bbox_x = slice(int(np.floor(coords[1]-size/2)), int(np.ceil(coords[1]+size/2)))
bbox_y = slice(int(np.floor(coords[0]-size/2)), int(np.ceil(coords[0]+size/2)))
cut = img_bgsub[test_t, bbox_y, bbox_x]

# Show
plt.imshow(cut, cmap='gray')
plt.show()

In [ ]:
# Generate all bounding boxes

median_sigma = np.median(np.concatenate(blobs)[:,2])
pct90_sigma = np.percentile(np.concatenate(blobs)[:,2], 90)

bbox_dict = {'median_sigma' : [],
             'pct90_sigma'  : [],
             'indiv_sigma'  : []}

for t in range(img.shape[0]):
    
    bboxes_median = []
    bboxes_pct90  = []
    bboxes_indiv  = []
    
    for spot in range(blobs[t].shape[0]):
        
        coords = blobs[t][spot][:2]
        
        size = 3 * median_sigma
        bbox_x = slice(int(np.floor(coords[1]-size/2)), int(np.ceil(coords[1]+size/2)))
        bbox_y = slice(int(np.floor(coords[0]-size/2)), int(np.ceil(coords[0]+size/2)))
        bboxes_median.append((bbox_y, bbox_x))
        
        size = 3 * pct90_sigma
        bbox_x = slice(int(np.floor(coords[1]-size/2)), int(np.ceil(coords[1]+size/2)))
        bbox_y = slice(int(np.floor(coords[0]-size/2)), int(np.ceil(coords[0]+size/2)))
        bboxes_pct90.append((bbox_y, bbox_x))
        
        size = 3 * blobs[t][spot][2]
        bbox_x = slice(int(np.floor(coords[1]-size/2)), int(np.ceil(coords[1]+size/2)))
        bbox_y = slice(int(np.floor(coords[0]-size/2)), int(np.ceil(coords[0]+size/2)))
        bboxes_indiv.append((bbox_y, bbox_x))
        
    bbox_dict['median_sigma'].append(bboxes_median)
    bbox_dict['pct90_sigma'].append(bboxes_pct90)
    bbox_dict['indiv_sigma'].append(bboxes_indiv)

#### Measure Brightness

In [ ]:
# Measure median and total brightness in bboxes

for t in range(img.shape[0]):
    
    brightness_median = []
    brightness_total  = []
    
    for spot in range(blobs[t].shape[0]):
        
        # For median brightness, use individual bboxes
        bbox_y, bbox_x = bbox_dict['indiv_sigma'][t][spot]
        cut = img[t, bbox_y, bbox_x]
        brightness_median.append(np.median(cut))
        
        # For total brightness, use median bboxes
        bbox_y, bbox_x = bbox_dict['median_sigma'][t][spot]
        cut = img[t, bbox_y, bbox_x]
        brightness_total.append(np.sum(cut))
    
    # Convert results to array
    blobs[t] = np.concatenate([blobs[t], np.array(brightness_median)[:, np.newaxis]], axis=-1)
    blobs[t] = np.concatenate([blobs[t], np.array(brightness_total)[:, np.newaxis]], axis=-1)

#### Measure Size (based on masking)

In [ ]:
# Measure size based on masking

# Aggregate pixels in bboxes for threshold calculation
aggregated = np.array([], dtype=np.uint8)
for t in range(img.shape[0]):
    for  spot in range(blobs[t].shape[0]):
        bbox_y, bbox_x = bbox_dict['pct90_sigma'][t][spot]
        cut = img_bgsub[t, bbox_y, bbox_x]
        aggregated = np.concatenate([aggregated, cut.flatten()])

# Compute threshold with Li method (looks decent in histogram)
thresh = threshold_li(aggregated)

# Show hist
plt.hist(aggregated, bins=256)
plt.vlines(thresh, ymin=0, ymax=10000, color='r', linewidth=2)
plt.ylim([0, 10000])
plt.show()

# Apply threshold and measure size
for t in range(img.shape[0]):    
    
    size_mask = []
    
    for spot in range(blobs[t].shape[0]):
        bbox_y, bbox_x = bbox_dict['pct90_sigma'][t][spot]
        cut = img_bgsub[t, bbox_y, bbox_x]
        mask = cut >= thresh
        size_mask.append(np.sum(mask))
    
    blobs[t] = np.concatenate([blobs[t], np.array(size_mask)[:, np.newaxis]], axis=-1)

### Check the Results

In [ ]:
# Number of spots

plt.figure(figsize=(10,5))

plt.plot(range(img.shape[0]), [blobs[t].shape[0] for t in range(img.shape[0])])

plt.title('spot count')
plt.xlabel('time [steps]')
plt.ylabel('number of detected SIM spots')

plt.show()

In [ ]:
# Brightness of spots (median)

plt.figure(figsize=(10,5))

# Plot every spot
for t in range(img.shape[0]):
    plt.scatter([t for _ in range(blobs[t].shape[0])], blobs[t][:,3], 
                c='teal', alpha=0.25, s=5)

# Plot median and mean
plt.plot(range(img.shape[0]), [np.median(blobs[t][:,3]) if blobs[t][:,3].size>0 else np.nan 
                               for t in range(img.shape[0])], 
         alpha=0.75, lw=2, label='median')
plt.plot(range(img.shape[0]), [np.mean(blobs[t][:,3]) if blobs[t][:,3].size>0 else np.nan 
                               for t in range(img.shape[0])], 
         alpha=0.75, lw=2, label='mean')

# Cosmetics
plt.legend()
plt.title('brightness (median)')
plt.xlabel('time [steps]')
plt.ylabel('brightness of detected SIM spots')

plt.show()

In [ ]:
# Brightness of spots (sum)

plt.figure(figsize=(10,5))

# Plot every spot
for t in range(img.shape[0]):
    plt.scatter([t for _ in range(blobs[t].shape[0])], blobs[t][:,4], 
                c='teal', alpha=0.25, s=5)

# Plot median and mean
plt.plot(range(img.shape[0]), [np.median(blobs[t][:,4]) if blobs[t][:,4].size>0 else np.nan 
                               for t in range(img.shape[0])], 
         alpha=0.75, lw=2, label='median')
plt.plot(range(img.shape[0]), [np.mean(blobs[t][:,4]) if blobs[t][:,4].size>0 else np.nan 
                               for t in range(img.shape[0])], 
         alpha=0.75, lw=2, label='mean')

# Cosmetics
plt.legend()
plt.title('brightness (sum)')
plt.xlabel('time [steps]')
plt.ylabel('brightness of detected SIM spots')

plt.show()

In [ ]:
# Size of spots (mask)

plt.figure(figsize=(10,5))

# Plot every spot
for t in range(img.shape[0]):
    plt.scatter([t for _ in range(blobs[t].shape[0])], blobs[t][:,5], 
                c='teal', alpha=0.25, s=5)
    
# Plot median and mean
plt.plot(range(img.shape[0]), [np.median(blobs[t][:,5]) if blobs[t][:,5].size>0 else np.nan 
                               for t in range(img.shape[0])], 
         alpha=0.75, lw=2, label='median')
plt.plot(range(img.shape[0]), [np.mean(blobs[t][:,5]) if blobs[t][:,5].size>0 else np.nan 
                               for t in range(img.shape[0])], 
         alpha=0.75, lw=2, label='mean')

# Cosmetics
plt.legend()
plt.title('size (mask)')
plt.xlabel('time [steps]')
plt.ylabel('numer of foreground pixels per detected SIM spot')

plt.show()